In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from scipy import signal

import os
import librosa as lb

import IPython.display as ipd

from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks

## Data Path

In [2]:
data_path = 'E:/REM Detection/psg_audio_dataset/testing_input_audio/'

In [3]:
def get_folder_names(data_path):
    folders = []
    for folder in os.listdir(data_path):
        folders.append(folder)
    return folders

def get_patient_id(folder_name):
    return int(folder_name)

def get_file_path_list(data_path, folder_name):
    file_path_list = []
    for file in os.listdir(data_path + folder_name):
        file_path_list.append(data_path + folder_name + '/' + file)
    return file_path_list

def get_file_name(file_path):
    return file_path.split('/')[-1].split('.')[0]


In [4]:
folders = get_folder_names(data_path)
patient_id = [get_patient_id(folder) for folder in folders]

file_path_list = []
for folder in folders:
    current_file_path_list = get_file_path_list(data_path, folder)
    file_path_list.extend(current_file_path_list)

print('Number of folders: ', len(folders))
print('Patient_id list: ', patient_id)
print('Number of files: ', len(file_path_list))

Number of folders:  2
Patient_id list:  [995, 999]
Number of files:  3270


In [5]:
file_id_list = [get_file_name(file_path) for file_path in file_path_list]

## Load Data

In [6]:
sample_rate = 44100

high_pass = signal.firwin(101, 1000, pass_zero= 'highpass', fs=sample_rate)

def apply_high_pass_filter(sig_array):
    return signal.lfilter(high_pass, [1.0], sig_array)

def apply_log_compression(sig_array, gamma):
    sign = np.sign(sig_array)
    abs_signal = 1 + np.abs(sig_array) * gamma
    logged = np.log(abs_signal)
    scaled = logged * (1 / np.log(1.0 + gamma))
    return sign * scaled

def nomalize_volume(sig_array):
    minAmp, maxAmp = np.amin(sig_array), np.amax(sig_array)
    max_energy = max(abs(minAmp), abs(maxAmp))
    scale = 1.0 / max_energy
    sig_array *= scale
    return sig_array

window_size_sec = 0.05
window_size = int(window_size_sec * sample_rate)

def get_spectrogram(sig_array):
    spectrograms = []
    for sig in sig_array:
        f, t, Sxx = signal.spectrogram(sig, sample_rate, nperseg=window_size)
        spectrograms.append((f, t, Sxx))
        if len(spectrograms) % 500 == 0:
            print('Loaded {} spectrograms'.format(len(spectrograms)))
            print('Remaining spectrograms: ', len(sig_array) - len(spectrograms))
    print('Number of spectrograms: ', len(spectrograms))
    return spectrograms

def load_and_process_audio(audio_file_list, sample_rate):
    output_audio = []
    for file in audio_file_list:
        audio, _ = lb.load(file, sr=sample_rate)
        audio = apply_high_pass_filter(audio)
        audio = apply_log_compression(audio, 18)
        audio = nomalize_volume(audio)
        output_audio.append(audio)
        if len(output_audio) % 500 == 0:
            print('Loaded {} audio files'.format(len(output_audio)))
            print('Remaining audio files: ', len(audio_file_list) - len(output_audio))
    print('Number of audio files: ', len(output_audio))
    return output_audio

In [7]:
processed_audio = load_and_process_audio(file_path_list, sample_rate)
spectrograms = get_spectrogram(processed_audio)

Loaded 500 audio files
Remaining audio files:  2770
Loaded 1000 audio files
Remaining audio files:  2270
Loaded 1500 audio files
Remaining audio files:  1770
Loaded 2000 audio files
Remaining audio files:  1270
Loaded 2500 audio files
Remaining audio files:  770
Loaded 3000 audio files
Remaining audio files:  270
Number of audio files:  3270
Loaded 500 spectrograms
Remaining spectrograms:  2770
Loaded 1000 spectrograms
Remaining spectrograms:  2270
Loaded 1500 spectrograms
Remaining spectrograms:  1770
Loaded 2000 spectrograms
Remaining spectrograms:  1270
Loaded 2500 spectrograms
Remaining spectrograms:  770
Loaded 3000 spectrograms
Remaining spectrograms:  270
Number of spectrograms:  3270


### visualize

In [8]:
def plot_audio(audio, processed_audio, sample_idx):
    fig, axs = plt.subplots(1, 2, figsize=(16, 5))
    fig.suptitle('Before and after noise removal', fontsize=16)

    axs[0].plot(audio[sample_idx])
    axs[0].set_title('Before noise removal')

    axs[1].plot(processed_audio[sample_idx])
    axs[1].set_title('After noise removal')

    for ax in axs:
        ax.set_xlabel('Time (samples)')
        ax.set_ylabel('Amplitude')

    plt.tight_layout()
    plt.show()

    print('Before noise removal: ', audio[sample_idx].shape)
    ipd.display(ipd.Audio(audio[sample_idx], rate=sample_rate))
    print('After noise removal: ', processed_audio[sample_idx].shape)
    ipd.display(ipd.Audio(processed_audio[sample_idx], rate=sample_rate))

def plot_spectrogram(spectrograms, sample_idx):
    spectrogram = spectrograms[sample_idx]
    plt.subplots(figsize=(16, 5))
    plt.pcolormesh(spectrogram[1], spectrogram[0], np.power(spectrogram[2], 0.1), shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

def peak_plot(power_envelope, time, peaks_idx, number_of_peaks):
    peak_timing = [time[peak] for peak in peaks_idx]
    plt.subplots(figsize=(16, 5))
    plt.plot(time, power_envelope, color='gray')
    plt.plot(peak_timing, power_envelope[peaks_idx], "v", color='red', markersize=10)
    plt.hlines(y=0.00008, xmin=0, xmax=time[-1], color='red', linestyles='dashed')
    plt.title('Number of peaks: ' + str(number_of_peaks))
    plt.ylabel('Power')
    plt.xlabel('Time [sec]')
    plt.show()

In [9]:
# sample_idx = 0

# plot_audio(audio_raw_signals, processed_audio, sample_idx)

In [10]:
# plot_spectrogram(spectrograms, sample_idx)

## get envelope

In [11]:
def get_envelope(spectrogram):
    times = []
    power_envelope = []
    for spec in spectrogram:
        times.append(spec[1])
        power_envelope.append(gaussian_filter1d(np.sum(spec[2], axis=0), sigma=10))
        if len(power_envelope) % 500 == 0:
            print('Loaded {} power envelopes'.format(len(power_envelope)))
            print('Remaining power envelopes: ', len(spectrogram) - len(power_envelope))
    print('Number of time series: ', len(times))
    print('Number of power envelopes: ', len(power_envelope))
    return times, power_envelope

def get_peaks_idx(power_envelope):
    peaks, _ = find_peaks(power_envelope, height=0.00008)
    return peaks

def get_peak_timing(time, power_envelope):
    peaks = get_peaks_idx(power_envelope)
    peak_timing = []
    for peak in peaks:
        peak_timing.append(time[peak])
    return peak_timing

In [12]:
times, power_envelopes = get_envelope(spectrograms)

peak_counts = []

for i in range(len(times)):
    peaks_idx = get_peaks_idx(power_envelopes[i])
    peak_counts.append(len(peaks_idx))

Loaded 500 power envelopes
Remaining power envelopes:  2770
Loaded 1000 power envelopes
Remaining power envelopes:  2270
Loaded 1500 power envelopes
Remaining power envelopes:  1770
Loaded 2000 power envelopes
Remaining power envelopes:  1270
Loaded 2500 power envelopes
Remaining power envelopes:  770
Loaded 3000 power envelopes
Remaining power envelopes:  270
Number of time series:  3270
Number of power envelopes:  3270


In [13]:
# peak plot
# sample_peaks = get_peaks_idx(power_envelopes[sample_idx])
# peak_plot(power_envelopes[sample_idx], times[sample_idx], sample_peaks, len(sample_peaks))

In [14]:
peak_df = pd.DataFrame({'sliced_audio_file_id': file_id_list, 'peak_algorithm': peak_counts})

In [15]:
# peak_df.to_csv('./data/peak_algorithm_mini.csv', index=False)

In [16]:
def get_patient_id_from_file_id(file_id):
    return int(file_id.split('_')[0])

In [17]:
peak_df['patient_id'] = peak_df['sliced_audio_file_id'].apply(get_patient_id_from_file_id)
peak_df

,sliced_audio_file_id,peak_algorithm,patient_id
0,00000995_01_000_000,0,995
1,00000995_01_000_001,6,995
2,00000995_01_000_002,0,995
3,00000995_01_000_003,0,995
4,00000995_01_000_004,7,995
...,...,...,...
3265,00000999_05_127_025,6,999
3266,00000999_05_127_026,10,999
3267,00000999_05_127_027,7,999
3268,00000999_05_127_028,2,999


In [18]:
peak_df.to_csv('./data/peak_algorithm_mini.csv', index=False)